In [ ]:
import time
import pickle
import numpy as np
from src.environment import StockTradingEnvironment, make_env
from src.utils import save_pickle, load_pickle, plot_grid
import gymnasium as gym
import gym_trading_env
import yfinance as yf
import random
import matplotlib.pyplot as plt
from collections import deque
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
import keras.backend as K
from keras.optimizers import Adam
from tensorflow.keras import models, layers, optimizers
from tensorflow.keras.callbacks import LearningRateScheduler
import logging
import multiprocessing as mp

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    device = '/GPU:0'
    logging.info("Using GPU for training")
else:
    device = '/CPU:0'
    logging.info("Using CPU for training")

In [ ]:
def fetch_stock_data(symbol, start_date, end_date, output_file):
    stock_data = yf.download(symbol, start=start_date, end=end_date)
    stock_data['Close'] = stock_data['Adj Close']
    
    stock_data = stock_data.drop(columns=['Adj Close'])

    # Convert column headers to lowercase
    stock_data.columns = stock_data.columns.str.lower()

    stock_data.to_csv(output_file)
        
    return stock_data

stock_data = fetch_stock_data('AAPL', '2019-06-20', '2024-06-21', 'AAPL_data.csv')

In [ ]:
def huber_loss(y_true, y_pred, clip_delta=1.0):
    error = y_true - y_pred
    cond = K.abs(error) <= clip_delta
    squared_loss = 0.5 * K.square(error)
    quadratic_loss = 0.5 * K.square(clip_delta) + clip_delta * (K.abs(error) - clip_delta)
    return K.mean(tf.where(cond, squared_loss, quadratic_loss))

#### Q-Learning

In [ ]:
class SumTree:
    def __init__(self, capacity):
        self.capacity = capacity
        self.tree = np.zeros(2 * capacity - 1)
        self.data = np.zeros(capacity, dtype=object)
        self.write = 0

    def _propagate(self, idx, change):
        parent = (idx - 1) // 2
        self.tree[parent] += change
        if parent != 0:
            self._propagate(parent, change)

    def _retrieve(self, idx, s):
        left = 2 * idx + 1
        right = left + 1
        if left >= len(self.tree):
            return idx
        if s <= self.tree[left]:
            return self._retrieve(left, s)
        else:
            return self._retrieve(right, s - self.tree[left])

    def total(self):
        return self.tree[0]

    def add(self, p, data):
        idx = self.write + self.capacity - 1
        self.data[self.write] = data
        self.update(idx, p)
        self.write += 1
        if self.write >= self.capacity:
            self.write = 0

    def update(self, idx, p):
        change = p - self.tree[idx]
        self.tree[idx] = p
        self._propagate(idx, change)

    def get(self, s):
        idx = self._retrieve(0, s)
        dataIdx = idx - self.capacity + 1
        return idx, self.tree[idx], self.data[dataIdx]

class PrioritizedReplayBuffer:
    def __init__(self, capacity, alpha=0.6):
        self.tree = SumTree(capacity)
        self.alpha = alpha
        self.capacity = capacity
        self.epsilon = 0.01

    def add(self, error, sample):
        p = (error + self.epsilon) ** self.alpha
        self.tree.add(p, sample)

    def sample(self, batch_size, beta=0.4):
        batch = []
        idxs = []
        segment = self.tree.total() / batch_size
        priorities = []
        for i in range(batch_size):
            a = segment * i
            b = segment * (i + 1)
            s = random.uniform(a, b)
            idx, p, data = self.tree.get(s)
            batch.append(data)
            idxs.append(idx)
            priorities.append(p)
        sampling_probabilities = priorities / self.tree.total()
        is_weights = np.power(self.tree.capacity * sampling_probabilities, -beta)
        is_weights /= is_weights.max()
        return batch, idxs, is_weights

    def update(self, idx, error):
        p = (error + self.epsilon) ** self.alpha
        self.tree.update(idx, p)

In [ ]:
class DoubleQLearningAgent:
    def __init__(self, env, learning_rate, discount_factor, epsilon=1.0, epsilon_min=0.01, epsilon_decay=0.995, bins=10):
        self.env = env
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.epsilon = epsilon
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_decay
        self.q_table1 = {}
        self.q_table2 = {}
        self.bins = bins

        if isinstance(env.single_action_space, gym.spaces.Discrete):
            self.action_space_type = 'Discrete'
            self.action_space_size = env.single_action_space.n
        elif isinstance(env.single_action_space, gym.spaces.MultiDiscrete):
            self.action_space_type = 'MultiDiscrete'
            self.action_space_size = env.single_action_space.nvec
        else:
            raise ValueError("Unsupported action space type")

    def discretize_state(self, state):
        discretized_state = tuple((state * self.bins).astype(int))
        return discretized_state

    def step(self, state):
        observation = state["observation"]
        state_index = self.discretize_state(observation)
        if np.random.rand() < self.epsilon:
            if self.action_space_type == 'Discrete':
                return np.random.randint(self.action_space_size)
            elif self.action_space_type == 'MultiDiscrete':
                return [np.random.randint(n) for n in self.action_space_size]
        else:
            q_values1 = self.q_table1.get(state_index, np.zeros(self.action_space_size))
            q_values2 = self.q_table2.get(state_index, np.zeros(self.action_space_size))
            q_values = q_values1 + q_values2
            if self.action_space_type == 'Discrete':
                return np.argmax(q_values)
            elif self.action_space_type == 'MultiDiscrete':
                return [np.argmax(q) for q in q_values]

    def update_qvalue(self, state, action, reward, next_state):
        observation = state["observation"]
        next_observation = next_state["observation"]
        state_index = self.discretize_state(observation)
        next_state_index = self.discretize_state(next_observation)
        if self.action_space_type == 'Discrete':
            if np.random.rand() < 0.5:
                best_next_action = np.argmax(self.q_table1.get(next_state_index, np.zeros(self.action_space_size)))
                q_value_update = reward + self.discount_factor * self.q_table2.get(next_state_index, np.zeros(self.action_space_size))[best_next_action]
                self.q_table1[state_index] = self.q_table1.get(state_index, np.zeros(self.action_space_size))
                self.q_table1[state_index][action] += self.learning_rate * (q_value_update - self.q_table1[state_index][action])
            else:
                best_next_action = np.argmax(self.q_table2.get(next_state_index, np.zeros(self.action_space_size)))
                q_value_update = reward + self.discount_factor * self.q_table1.get(next_state_index, np.zeros(self.action_space_size))[best_next_action]
                self.q_table2[state_index] = self.q_table2.get(state_index, np.zeros(self.action_space_size))
                self.q_table2[state_index][action] += self.learning_rate * (q_value_update - self.q_table2[state_index][action])
        elif self.action_space_type == 'MultiDiscrete':
            if np.random.rand() < 0.5:
                best_next_action = [np.argmax(q) for q in self.q_table1.get(next_state_index, np.zeros(self.action_space_size))]
                q_values2_next = self.q_table2.get(next_state_index, np.zeros(self.action_space_size))
                update_value = reward + self.discount_factor * sum(q_values2_next[i, best_next_action[i]] for i in range(len(best_next_action)))
                self.q_table1[state_index] = self.q_table1.get(state_index, np.zeros(self.action_space_size))
                for i in range(len(action)):
                    self.q_table1[state_index][i, action[i]] += self.learning_rate * (update_value - self.q_table1[state_index][i, action[i]])
            else:
                best_next_action = [np.argmax(q) for q in self.q_table2.get(next_state_index, np.zeros(self.action_space_size))]
                q_values1_next = self.q_table1.get(next_state_index, np.zeros(self.action_space_size))
                update_value = reward + self.discount_factor * sum(q_values1_next[i, best_next_action[i]] for i in range(len(best_next_action)))
                self.q_table2[state_index] = self.q_table2.get(state_index, np.zeros(self.action_space_size))
                for i in range(len(action)):
                    self.q_table2[state_index][i, action[i]] += self.learning_rate * (update_value - self.q_table2[state_index][i, action[i]])

    def decay_epsilon(self):
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

# DQN Agent
class DQNAgent:
    def __init__(self, env, model_name='DQN_model', gamma=0.95, epsilon=1.0, epsilon_min=0.01, epsilon_decay=0.995, learning_rate=0.001):
        self.env = env
        self.model_name = model_name
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_decay
        self.learning_rate = learning_rate
        self.loss = huber_loss
        self.custom_objects = {"huber_loss": huber_loss}
        self.optimizer = Adam(learning_rate=self.learning_rate)
        self.buffer_size = 10000
        self.batch_size = 32
        self.memory = PrioritizedReplayBuffer(self.buffer_size, 0.6)
        self.beta = 0.4
        self.model = self.build_model()
        self.target_model = self.build_model()
        self.update_target_model()

    def build_model(self):
        model = Sequential()
        model.add(Dense(units=128, activation="relu", input_dim=self.env.observation_space.shape[1]))
        model.add(Dense(units=256, activation="relu"))
        model.add(Dense(units=256, activation="relu"))
        model.add(Dense(units=128, activation="relu"))
        model.add(Dense(units=self.env.action_space.n))
        model.compile(loss=self.loss, optimizer=self.optimizer)
        return model

    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())

    def remember(self, state, action, reward, next_state, done):
        state = state.flatten().reshape(1, -1)
        next_state = next_state.flatten().reshape(1, -1)
        target = reward + self.gamma * np.amax(self.target_model.predict(next_state)[0]) if not done else reward
        td_error = abs(target - np.amax(self.model.predict(state)[0]))
        self.memory.add(td_error, (state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return np.random.choice(self.env.action_space.n)
        state = state.flatten().reshape(1, -1)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])

    def replay(self):
        if len(self.memory.tree.data) < self.batch_size:
            return
        minibatch, idxs, is_weights = self.memory.sample(self.batch_size, self.beta)
        states, actions, rewards, next_states, dones = zip(*minibatch)
        states = np.array(states).reshape(self.batch_size, -1)
        next_states = np.array(next_states).reshape(self.batch_size, -1)
        targets = self.model.predict(states)
        targets_next = self.target_model.predict(next_states)

        for i in range(self.batch_size):
            target = rewards[i] + self.gamma * np.amax(targets_next[i]) if not dones[i] else rewards[i]
            td_error = abs(target - targets[i][actions[i]])
            self.memory.update(idxs[i], td_error)
            targets[i][actions[i]] = target

        self.model.fit(states, targets, epochs=1, verbose=0)
        self.update_target_model()

    def decay_epsilon(self):
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

In [ ]:
def q_learning_learning_loop(env, agent, learning_rate: float, discount_factor: float, episodes: int,
                             min_epsilon_allowed: float, initial_epsilon_value: float,
                             buffer_size: int = 10000, batch_size: int = 32, decay_method="exponential") -> tuple:
    epsilon = initial_epsilon_value
    epsilon_decay_factor = np.power(min_epsilon_allowed / epsilon, 1 / episodes)

    reward_across_episodes = []
    epsilons_across_episodes = []

    replay_buffer = deque(maxlen=buffer_size)

    for episode in range(episodes):
        observations, _ = env.reset()
        terminated = np.array([False] * env.num_envs)
        truncated = np.array([False] * env.num_envs)
        episode_rewards = np.zeros(env.num_envs)
        
        while not np.all(terminated) and not np.all(truncated):
            actions = [agent.step({"observation": observation}) for observation in observations]
            next_observations, rewards, terminated, truncated, _ = env.step(actions)
            episode_rewards += rewards
            
            for obs, action, reward, next_obs, term, trun in zip(observations, actions, rewards, next_observations, terminated, truncated):
                replay_buffer.append(({"observation": obs}, action, reward, {"observation": next_obs}, term or trun))
                
            observations = next_observations

            if len(replay_buffer) >= batch_size:
                batch = np.array(random.sample(replay_buffer, batch_size), dtype=object)
                for b_state, b_action, b_reward, b_next_state, b_done in batch:
                    b_state = agent.discretize_state(b_state["observation"])
                    b_next_state = agent.discretize_state(b_next_state["observation"])
                    if b_done:
                        target = b_reward
                    else:
                        if isinstance(env.single_action_space, gym.spaces.Discrete):
                            target = b_reward + discount_factor * np.max(agent.q_table1.get(b_next_state, np.zeros(env.single_action_space.n)) + agent.q_table2.get(b_next_state, np.zeros(env.single_action_space.n)))
                        elif isinstance(env.single_action_space, gym.spaces.MultiDiscrete):
                            target = b_reward + discount_factor * np.max(agent.q_table1.get(b_next_state, np.zeros(env.single_action_space.nvec)) + agent.q_table2.get(b_next_state, np.zeros(env.single_action_space.nvec)))
                    
                    if isinstance(env.single_action_space, gym.spaces.Discrete):
                        agent.q_table1[b_state] = agent.q_table1.get(b_state, np.zeros(env.single_action_space.n))
                    elif isinstance(env.single_action_space, gym.spaces.MultiDiscrete):
                        agent.q_table1[b_state] = agent.q_table1.get(b_state, np.zeros(env.single_action_space.nvec))
                    
                    agent.q_table1[b_state][b_action] += learning_rate * (target - agent.q_table1[b_state][b_action])

        if decay_method == "exponential":
            epsilon = max(min_epsilon_allowed, epsilon * epsilon_decay_factor)
        else:
            epsilon = max(min_epsilon_allowed, epsilon - (initial_epsilon_value - min_epsilon_allowed) / episodes)

        reward_across_episodes.append(np.mean(episode_rewards))
        epsilons_across_episodes.append(epsilon)

        agent.decay_epsilon()

        if (episode + 1) % 100 == 0:
            logging.info(f"Episode {episode + 1}/{episodes} - Mean Reward: {np.mean(episode_rewards)} - Epsilon: {epsilon}")

    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(reward_across_episodes, label='Rewards')
    plt.xlabel('Episodes')
    plt.ylabel('Mean Reward')
    plt.title('Rewards over Episodes')
    plt.legend()
    plt.grid(True)

    plt.subplot(1, 2, 2)
    plt.plot(epsilons_across_episodes, label='Epsilon')
    plt.xlabel('Episodes')
    plt.ylabel('Epsilon')
    plt.title('Epsilon Decay over Episodes')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.show()

    return agent, reward_across_episodes, epsilons_across_episodes

In [ ]:
num_envs = 3
file_path = 'AAPL_data.csv'
envs = gym.vector.SyncVectorEnv([lambda: make_env(file_path, number_of_days_to_consider=20, n_select=15) for _ in range(num_envs)])

# Initialize Agent and Train
agent = DoubleQLearningAgent(envs, learning_rate=0.001, discount_factor=0.95)

agent, reward_across_episodes, epsilons_across_episodes = q_learning_learning_loop(
    envs,
    agent,
    learning_rate=0.001,
    discount_factor=0.95,
    episodes=40000,
    min_epsilon_allowed=0.01,
    initial_epsilon_value=1,
    buffer_size=10000,
    batch_size=128,
    decay_method="exponential"
)

In [ ]:
def run_learned_policy(env, agent):
    obs = env.reset()
    terminated, truncated = np.array([False] * env.num_envs), np.array([False] * env.num_envs)
    total_rewards = np.zeros(env.num_envs)

    while not np.all(terminated) and not np.all(truncated):
        actions = [np.argmax(agent.q_table1.get(tuple(observation.flatten()), np.zeros(env.action_space.n)) + agent.q_table2.get(tuple(observation.flatten()), np.zeros(env.action_space.n))) for observation in obs]
        obs, rewards, terminated, truncated, _ = env.step(actions)
        total_rewards += rewards

    logging.info("Total Rewards: %s", total_rewards)
    return total_rewards

total_rewards = run_learned_policy(envs, agent)

In [ ]:
def plot_grid(env, agent, reward_across_episodes: list, epsilons_across_episodes: list) -> None:
    env.train = False
    total_reward_learned_policy = [run_learned_policy(env, agent) for _ in range(30)]

    plt.figure(figsize=(15, 10))

    plt.subplot(2, 2, 1)
    plt.plot(reward_across_episodes, 'ro')
    plt.xlabel('Episode')
    plt.ylabel('Reward Value')
    plt.title('Rewards Per Episode (Training)')
    plt.grid()

    plt.subplot(2, 2, 2)
    plt.plot(total_reward_learned_policy, 'ro')
    plt.xlabel('Episode')
    plt.ylabel('Reward Value')
    plt.title('Rewards Per Episode (Learned Policy Evaluation)')
    plt.grid()

    plt.subplot(2, 2, 3)
    plt.plot(reward_across_episodes)
    plt.xlabel('Episode')
    plt.ylabel('Cumulative Reward Per Episode (Training)')
    plt.title('Cumulative Reward vs Episode')
    plt.grid()

    plt.subplot(2, 2, 4)
    plt.plot(epsilons_across_episodes)
    plt.xlabel('Episode')
    plt.ylabel('Epsilon Values')
    plt.title('Epsilon Decay')
    plt.grid()

    plt.tight_layout()
    plt.show()

plot_grid(envs, agent, reward_across_episodes, epsilons_across_episodes)

In [ ]:
stock_trading_environment = StockTradingEnvironment('./AAPL_data.csv', number_of_days_to_consider=30)
stock_trading_environment.train = False
obs, _ = stock_trading_environment.reset()
terminated, truncated = False, False
while not terminated:
    # Convert observation to a tuple
    obs_tuple = tuple(obs.flatten())
    action = np.argmax(agent.q_table1.get(obs_tuple, np.zeros(envs.action_space.n)) + agent.q_table2.get(obs_tuple, np.zeros(envs.action_space.n)))
    obs, reward, terminated, truncated, info = stock_trading_environment.step(action)

stock_trading_environment.render()

In [ ]:
save_pickle(agent, 'aapl_q_learning_agent.pkl')

#### Forecast

In [ ]:
agent = load_pickle("aapl_q_learning_agent.pkl")

In [ ]:
stock_trading_environment = make_env('AAPL_data.csv', number_of_days_to_consider=30)
stock_trading_environment.train = False
obs, _ = stock_trading_environment.reset()
terminated, truncated = False, False
while not terminated:
    obs_tuple = tuple(obs.flatten())
    action = np.argmax(agent.q_table1.get(obs_tuple, np.zeros(envs.action_space.n)) + agent.q_table2.get(obs_tuple, np.zeros(envs.action_space.n)))
    obs, reward, terminated, truncated, info = stock_trading_environment.step(action)

stock_trading_environment.render()